<div id="top">
<!-- <div style="background-image: url(https://www.emtmadrid.es/getattachment/da3be644-cb9d-44db-8011-e3f40f1c5c34); opacity: 0.2"/> -->
<img src="https://www.gmv.com/sites/default/files/content/image/2021/11/03/115/gmv_rgbredblack.png" alt="GMV Logo" style="width: 200px">
<img src="https://www.upm.es/sfs/Rectorado/Gabinete%20del%20Rector/Logos/UPM/CEI/LOGOTIPO%20leyenda%20color%20JPG%20p.png" alt="UPM Logo" style="float: right; width: 200px">
<h1><b>QA: AEMET dataset 🌥️</b></h1>
<h5 style="text-align: right">INESDATA-MOV</h5>
</div>

# Análisis de calidad
Este cuaderno analiza la calidad del dataset proveniente de la fuente de datos de la Agencia Estatal de Meteorología ([AEMET](https://www.aemet.es/es/portada)). La calidad del mismo se validará teniendo en cuenta los siguientes aspectos:
* Análisis de las variables
* Conversiones de tipos de datos
* Checks de calidad del dato
* Análisis Exploratorio de los datos (EDA)

La **calidad del dato** se refiere a la medida en que los datos son adecuados para su uso, por lo que es esencial para garantizar la confiabilidad y utilidad de los datos en diversas aplicaciones y contextos. Así, en este notebook se evaluarán también las cinco dimensiones de la calidad del dato:
1. **Unicidad**: Ausencia de duplicados o registros repetidos en un conjunto de datos. Los datos son únicos cuando cada registro o entidad en el conjunto de datos es único y no hay duplicados presentes.
2. **Exactitud**: Los datos exactos son libres de errores y representan con precisión la realidad que están destinados a describir. Esto implica que los datos deben ser correctos y confiables para su uso en análisis y toma de decisiones.
3. **Completitud**: Los datos completos contienen toda la información necesaria para el análisis y no tienen valores faltantes o nulos que puedan afectar la interpretación o validez de los resultados.
4. **Consistencia**: Los datos consistentes mantienen el mismo formato, estructura y significado en todas las instancias, lo que facilita su comparación y análisis sin ambigüedad.
5. **Validez**: Medida en que los datos son precisos y representan con exactitud la realidad que están destinados a describir. 

<div class="admonition info">
<p class="admonition-title">Nota</p>
<p>
Este dataset ha sido creado ejecutando el comando <code>create</code> del paquete de Python <a href="https://github.com/oeg-upm/inesdata-mov-data-generation"><code>inesdata_mov_datasets</code></a>.<br>
Para poder ejecutar este comando es necesario haber ejecutado antes el comando <code>extract</code>, que realiza la extracción de datos de la API de la AEMET y los almacena en Minio. El comando <code>create</code> se encargaría de descargar dichos datos y unirlos todos en un único dataset.
</p>
</div>

In [1]:
import os
from datetime import datetime

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from ydata_profiling import ProfileReport

sns.set_palette("deep")
import warnings

warnings.filterwarnings("ignore")

In [2]:
ROOT_PATH = os.path.dirname(os.path.dirname(os.path.abspath(os.getcwd())))
DATA_PATH = os.path.join(ROOT_PATH, "data", "processed")
AEMET_DATA_PATH = os.path.join(DATA_PATH, "aemet")
AEMET_DATA_PATH

'/home/code/inesdata-mov/data-generation/data/processed/aemet'

In [3]:
for w in os.walk(AEMET_DATA_PATH):
    print(w)

('/home/code/inesdata-mov/data-generation/data/processed/aemet', ['2024'], [])
('/home/code/inesdata-mov/data-generation/data/processed/aemet/2024', ['03'], [])
('/home/code/inesdata-mov/data-generation/data/processed/aemet/2024/03', ['06', '07', '12', '02', '05', '11', '04', '03', '08', '13'], [])
('/home/code/inesdata-mov/data-generation/data/processed/aemet/2024/03/06', [], ['aemet_20240306.csv'])
('/home/code/inesdata-mov/data-generation/data/processed/aemet/2024/03/07', [], ['aemet_20240307.csv'])
('/home/code/inesdata-mov/data-generation/data/processed/aemet/2024/03/12', [], ['aemet_20240312.csv'])
('/home/code/inesdata-mov/data-generation/data/processed/aemet/2024/03/02', [], ['aemet_20240302.csv'])
('/home/code/inesdata-mov/data-generation/data/processed/aemet/2024/03/05', [], ['aemet_20240305.csv'])
('/home/code/inesdata-mov/data-generation/data/processed/aemet/2024/03/11', [], ['aemet_20240311.csv'])
('/home/code/inesdata-mov/data-generation/data/processed/aemet/2024/03/04', 

**Cada fila de este dataset representa la meteorología de Madrid para una determinada fecha y hora concretos.**

<div class="admonition warning">
<p class="admonition-title">-</p>
<p>
Vamos a analizar la calidad del dataset generado solamente para el día 13 de marzo, en el futuro dispondremos de más días.
</p>
</div>

In [4]:
df = pd.read_csv(
    os.path.join(AEMET_DATA_PATH, "2024", "03", "13", "aemet_20240313.csv"),
    parse_dates=["datetime"],
)
df.head(10)

,estadoCielo_value,estadoCielo_descripcion,precipitacion_value,probPrecipitacion_value,probTormenta_value,nieve_value,probNieve_value,temperatura_value,sensTermica_value,humedadRelativa_value,direccion,velocidad,vientoAndRachaMax_value,datetime
0,17n,Nubes altas,0.0,NaN,NaN,0.0,NaN,8.0,6.0,68.0,['NE'],['10'],NaN,2024-03-13 00:00:00
1,17n,Nubes altas,0.0,NaN,NaN,0.0,NaN,8.0,6.0,68.0,NaN,NaN,18.0,2024-03-13 00:00:00
2,17n,Nubes altas,0.0,NaN,NaN,0.0,NaN,6.0,3.0,75.0,['NE'],['12'],NaN,2024-03-13 01:00:00
3,17n,Nubes altas,0.0,NaN,NaN,0.0,NaN,6.0,3.0,75.0,NaN,NaN,17.0,2024-03-13 01:00:00
4,NaN,NaN,NaN,0.0,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,2024-03-13 01:07:00
5,17n,Nubes altas,0.0,NaN,NaN,0.0,NaN,6.0,4.0,73.0,['NE'],['11'],NaN,2024-03-13 02:00:00
6,17n,Nubes altas,0.0,NaN,NaN,0.0,NaN,6.0,4.0,73.0,NaN,NaN,16.0,2024-03-13 02:00:00
7,12n,Poco nuboso,0.0,NaN,NaN,0.0,NaN,6.0,4.0,74.0,['NE'],['11'],NaN,2024-03-13 03:00:00
8,12n,Poco nuboso,0.0,NaN,NaN,0.0,NaN,6.0,4.0,74.0,NaN,NaN,17.0,2024-03-13 03:00:00
9,12n,Poco nuboso,0.0,NaN,NaN,0.0,NaN,6.0,3.0,75.0,['NE'],['12'],NaN,2024-03-13 04:00:00


In [5]:
df.shape

(52, 14)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52 entries, 0 to 51
Data columns (total 14 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   estadoCielo_value        48 non-null     object        
 1   estadoCielo_descripcion  48 non-null     object        
 2   precipitacion_value      48 non-null     float64       
 3   probPrecipitacion_value  4 non-null      float64       
 4   probTormenta_value       4 non-null      float64       
 5   nieve_value              48 non-null     float64       
 6   probNieve_value          4 non-null      float64       
 7   temperatura_value        48 non-null     float64       
 8   sensTermica_value        48 non-null     float64       
 9   humedadRelativa_value    48 non-null     float64       
 10  direccion                24 non-null     object        
 11  velocidad                24 non-null     object        
 12  vientoAndRachaMax_value  24 non-null  

In [7]:
df.columns

Index(['estadoCielo_value', 'estadoCielo_descripcion', 'precipitacion_value',
       'probPrecipitacion_value', 'probTormenta_value', 'nieve_value',
       'probNieve_value', 'temperatura_value', 'sensTermica_value',
       'humedadRelativa_value', 'direccion', 'velocidad',
       'vientoAndRachaMax_value', 'datetime'],
      dtype='object')

In [8]:
df["estadoCielo_descripcion"].value_counts()

estadoCielo_descripcion
Nubes altas    28
Despejado      14
Poco nuboso     6
Name: count, dtype: int64

## Conversiones de tipos

La variable `estadoCielo_value` tiene valores iguales pero terminados en 'n' que significan lo mismo. Además, indican la categoría descrita en la columna `estadoCielo_descripción`. Por tanto, esta columna puede ser eliminada, y la renombramos para que los nombres de las variables estén unificados. También la convertimos a categorías.

In [9]:
df["estadoCielo_value"].value_counts()

estadoCielo_value
17     16
17n    12
11n     8
11      6
12n     4
12      2
Name: count, dtype: int64

In [10]:
df.drop(columns='estadoCielo_value', inplace=True)
df.rename(columns={'estadoCielo_descripcion': 'estadoCielo_value'}, inplace=True)

df["estadoCielo_value"] = df["estadoCielo_value"].astype('category')
df["estadoCielo_value"].value_counts()

estadoCielo_value
Nubes altas    28
Despejado      14
Poco nuboso     6
Name: count, dtype: int64

La variable `direccion` representa los valores de norte, sur, este, oeste. Como vienen en una lista, los sacamos y los convertimos a categorías.

In [11]:
df["direccion"].value_counts()

direccion
['NE']    10
['E']      5
['S']      5
['NO']     3
['SE']     1
Name: count, dtype: int64

In [12]:
df["direccion"] = df["direccion"].str.replace("['", "").str.replace("']", "").astype('category')
df["direccion"].value_counts()

direccion
NE    10
E      5
S      5
NO     3
SE     1
Name: count, dtype: int64

Los valores de la variable `velocidad` vienen en una lista, los sacamos y los convertimos a `float`.

In [13]:
df['velocidad'].value_counts()

velocidad
['8']     5
['10']    4
['12']    3
['11']    3
['7']     3
['6']     2
['9']     2
['5']     1
['4']     1
Name: count, dtype: int64

In [14]:
df["velocidad"] = df["velocidad"].str.replace("['", "").str.replace("']", "").astype(float)
df["velocidad"].value_counts()

velocidad
8.0     5
10.0    4
12.0    3
11.0    3
7.0     3
6.0     2
9.0     2
5.0     1
4.0     1
Name: count, dtype: int64

Obtenemos los tipos de las variables:

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52 entries, 0 to 51
Data columns (total 13 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   estadoCielo_value        48 non-null     category      
 1   precipitacion_value      48 non-null     float64       
 2   probPrecipitacion_value  4 non-null      float64       
 3   probTormenta_value       4 non-null      float64       
 4   nieve_value              48 non-null     float64       
 5   probNieve_value          4 non-null      float64       
 6   temperatura_value        48 non-null     float64       
 7   sensTermica_value        48 non-null     float64       
 8   humedadRelativa_value    48 non-null     float64       
 9   direccion                24 non-null     category      
 10  velocidad                24 non-null     float64       
 11  vientoAndRachaMax_value  24 non-null     float64       
 12  datetime                 52 non-null  

In [16]:
num_cols = list(df.select_dtypes(include=np.number).columns)
cat_cols = list(df.select_dtypes(include=["category"]).columns)
date_cols = list(df.select_dtypes(exclude=[np.number, "category"]).columns)

print(f"Numeric cols: {num_cols}")
print(f"Categoric cols: {cat_cols}")
print(f"Date cols: {date_cols}")

Numeric cols: ['precipitacion_value', 'probPrecipitacion_value', 'probTormenta_value', 'nieve_value', 'probNieve_value', 'temperatura_value', 'sensTermica_value', 'humedadRelativa_value', 'velocidad', 'vientoAndRachaMax_value']
Categoric cols: ['estadoCielo_value', 'direccion']
Date cols: ['datetime']


## QA checks ✅

### Unicidad
Como hemos comentado anteriormente, **cada fila de este dataset representa la meteorología de Madrid para una determinada fecha y hora concretos.** Por tanto, las claves primarias de este dataset se conformarán teniendo en cuenta dichos atributos:

In [17]:
df['datetime'].nunique()

28

In [18]:
#  TODO

### Exactitud y Completitud

In [19]:
#  TODO

### Consistencia y Validez

In [20]:
#  TODO

## PROFILING 📑

In [21]:
profile = ProfileReport(
    df,
    title="🌥️ AEMET QA",
    dataset={
        "description": "AEMET - Estado de la meteorología",
        "url": "https://opendata.aemet.es/dist/index.html",
    },
    variables={
        "descriptions": {
            "PK": "Identificador único (Primary Key) del dataset, compuesto por <datetime>_TODO",
            "datetime": "Fecha y hora de la petición a la API",
            "estadoCielo_value": "Estado del cielo",
            "precipitacion_value": "Precipitación",
            "probPrecipitacion_value": "Probabilidad de precipitación",
            "probTormenta_value": "Probabilidad de tormenta",
            "nieve_value": "Nieve",
            "probNieve_value": "Probabilidad de nieve",
            "temperatura_value": "Temperatura",
            "sensTermica_value": "Sensación térmica",
            "humedadRelativa_value": "Humedad relativa",
            "direccion": "Dirección del viento",
            "velocidad": "Velocidad del viento",
            "vientoAndRachaMax_value": "Viento racha máxima",
        }
    },
    interactions=None,
    explorative=True,
    dark_mode=True,
)
profile.to_file(os.path.join(ROOT_PATH, "docs", "qa", "aemet_report.html"))
# profile.to_notebook_iframe()

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

## EDA 📊🔍

In [ ]:
#  TODO